<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Using AzureML Pipelines and AutoML for Predicting Sentence Similarity

This notebook demonstrates how to use AzureML pipelines and AutoML to streamline the creation of a machine learning workflow for predicting sentence similarity. The pipeline contains two steps:   
1. PythonScriptStep: embeds sentences using a popular sentence embedding model, Google Universal Sentence Encoder
2. AutoMLStep: demonstrates how to use AutoML to automate model selection for predicting sentence similarity (regression)

This notebook showcases how to use the following AzureML features:  
- AzureML Pipelines
- AutoML
- AmlCompute
- Datastore
- Logging

## Table of Contents
1. [Introduction](#1.-Introduction)  
    * 1.1 [What are AzureML Pipelines?](#1.1-What-are-AzureML-Pipelines?)  
    * 1.2 [What is Azure AutoML?](#1.2-What-is-Azure-AutoML?)  
    * 1.3 [Modeling Problem](#1.3-Modeling-Problem)  
    
    
2. [Data Preparation](#2.-Data-Preparation)  


3. [AzureML Setup](#3.-AzureML-Setup)  
    * 3.1 [Link to or create a `Workspace`](#3.1-Link-to-or-create-a-Workspace)  
    * 3.2 [Set up an `Experiment` and logging](#3.2-Set-up-an-Experiment-and-logging)  
    * 3.3 [Link `AmlCompute` compute target](#3.3-Link-AmlCompute-compute-target)  
    * 3.4 [Upload data to `Datastore`](#3.4-Upload-data-to-Datastore)  
    
    
4. [Create AzureML Pipeline](#4.-Create-AzureML-Pipeline)  
    * 4.1 [Set up run configuration file](#4.1-Set-up-run-configuration-file)  
    * 4.2 [PythonScriptStep](#4.2-PythonScriptStep)  
        * 4.2.1 [Define python script to run](#4.2.1-Define-python-script-to-run)
        * 4.2.2 [Create PipelineData object](#4.2.2-Create-PipelineData-object)
        * 4.2.3 [Create PythonScriptStep](#4.2.3-Create-PythonScriptStep)
        
    * 4.3 [AutoMLStep](#4.3-AutoMLStep)
        * 4.3.1 [Define get_data script to load data](#4.3.1-Define-get_data-script-to-load-data)
        * 4.3.2 [Create AutoMLConfig object](#4.3.2-Create-AutoMLConfig-object)
        * 4.3.3 [Create AutoMLStep](#4.3.3-Create-AutoMLStep)
        
        
5. [Run Pipeline](#5.-Run-Pipeline)

## 1. Introduction

### 1.1 What are AzureML Pipelines?

AzureML Pipelines "define reusable machine learning workflows that can be used as a template for your machine learning scenarios" ([pipeline information](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines)). Pipelines allow you to optimize your workflow and spend time on machine learning rather than infrastructure. A Pipeline is defined by a series of steps; the following steps are available: AdlaStep, AutoMLStep, AzureBatchStep, DataTransferStep, DatabricksStep, EstimatorStep, HyperDriveStep, ModuleStep, MpiStep, and PythonScriptStep (see [here](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/?view=azure-ml-py) for details of each step). When the pipeline is run, cached results are used for all steps that have not changed, optimizing the run time. Data sources and intermediate data can be used across multiple steps in a pipeline, saving time and resources. Below we see an example of an AzureML pipeline.

![](pipelines.png)

### 1.2 What is Azure AutoML?

Automated machine learning (AutoML) is a capability of Microsoft's Azure Machine Learning service. The goal of AutoML is to "improve the productivity of data scientists and democratize AI" [1] by allowing for the rapid development and deployment of machine learning models. To acheive this goal, AutoML automates the process of selecting a ML model and tuning the model. All the user is required to provide is a dataset (suitable for a classification, regression, or time-series forecasting problem) and a metric to optimize in choosing the model and hyperparameters. The user is also given the ability to set time and cost constraints for the model selection and tuning.

[1]https://azure.microsoft.com/en-us/blog/new-automated-machine-learning-capabilities-in-azure-machine-learning-service/

![](automl.png)

The AutoML model selection and tuning process can be easily tracked through the Azure portal or directly in python notebooks through the use of widgets. AutoML quickly selects a high quilty machine learning model tailored for your prediction problem. In this notebook, we walk through the steps of preparing data, setting up an AutoML experiment, and evaluating the results of our best model. More information about running AutoML experiments in Python can be found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train). 

### 1.3 Modeling Problem

The regression problem we will demonstrate is predicting sentence similarity scores on the STS Benchmark dataset. The [STS Benchmark dataset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark#STS_benchmark_dataset_and_companion_dataset) contains a selection of English datasets that were used in Semantic Textual Similarity (STS) tasks 2012-2017. The dataset contains 8,628 sentence pairs with a human-labeled integer representing the sentences' similarity (ranging from 0, for no meaning overlap, to 5, meaning equivalence).

For each sentence in the sentence pair, we will use Google's pretrained Universal Sentence Encoder (details provided below) to generate a $512$-dimensional embedding. Both embeddings in the sentence pair will be concatenated and the resulting $1024$-dimensional vector will be used as features in our regression problem. Our target variable is the sentence similarity score.

In [34]:
# Set the environment path to find NLP
import sys
sys.path.append("../../")
import time
import logging
import csv
import os
import pandas as pd
import shutil
import numpy as np
import torch
import sys
from scipy.stats import pearsonr
from scipy.spatial import distance
from sklearn.externals import joblib

# Import utils
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset import stsbenchmark
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)

# Tensorflow dependencies for Google Universal Sentence Encoder
import tensorflow as tf
import tensorflow_hub as hub
tf.logging.set_verbosity(tf.logging.ERROR) # reduce logging output

# AzureML packages
import azureml as aml
import logging
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)
from azureml.train.automl import AutoMLConfig
from azureml.core import Datastore, Experiment
from azureml.data.data_reference import DataReference  
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.automl import AutoMLStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import PythonScriptStep

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", aml.core.VERSION)
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow Version:", tf.VERSION)

Turning diagnostics collection on. 
System version: 3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)]
Azure ML SDK Version: 1.0.41
Pandas version: 0.23.4
Tensorflow Version: 1.13.1


In [35]:
BASE_DATA_PATH = '../../data'

# 2. Data Preparation

**STS Benchmark Dataset**

As described above, the STS Benchmark dataset contains 8.6K sentence pairs along with a human-annotated score for how similiar the two sentences are. We will load the training, development (validation), and test sets provided by STS Benchmark and preprocess the data (lowercase the text, drop irrelevant columns, and rename the remaining columns) using the utils contained in this repo. Each dataset will ultimately have three columns: _sentence1_ and _sentence2_ which contain the text of the sentences in the sentence pair, and _score_ which contains the human-annotated similarity score of the sentence pair.

In [36]:
# Load in the raw datasets as pandas dataframes
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")
dev_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="dev")
test_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="test")

100%|████████████████████████████████████████████████| 401/401 [00:02<00:00, 191KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|████████████████████████████████████████████████| 401/401 [00:01<00:00, 211KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|████████████████████████████████████████████████| 401/401 [00:01<00:00, 210KB/s]


Data downloaded to ../../data\raw\stsbenchmark


In [37]:
# Clean each dataset by lowercasing text, removing irrelevant columns,
# and renaming the remaining columns
train_clean = stsbenchmark.clean_sts(train_raw)
dev_clean = stsbenchmark.clean_sts(dev_raw)
test_clean = stsbenchmark.clean_sts(test_raw)

In [38]:
# Convert all text to lowercase
train = to_lowercase(train_clean)
dev = to_lowercase(dev_clean)
test = to_lowercase(test_clean)

In [39]:
print("Training set has {} sentences".format(len(train)))
print("Development set has {} sentences".format(len(dev)))
print("Testing set has {} sentences".format(len(test)))

Training set has 5749 sentences
Development set has 1500 sentences
Testing set has 1379 sentences


In [40]:
train.head(5)

,score,sentence1,sentence2
0,5.00,a plane is taking off.,an air plane is taking off.
1,3.80,a man is playing a large flute.,a man is playing a flute.
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...
3,2.60,three men are playing chess.,two men are playing chess.
4,4.25,a man is playing the cello.,a man seated is playing the cello.


In [41]:
#Save the cleaned data
if not os.path.isdir('data'):
    os.mkdir('data')
    
train.to_csv("data/train.csv", index=False)
test.to_csv("data/test.csv", index=False)
dev.to_csv("data/dev.csv", index=False)

# 3. AzureML Setup

Now, we set up the necessary components for running this as an AzureML experiment
1. Create or link to an existing `Workspace`
2. Set up an `Experiment` with `logging`
3. Create or attach existing `AmlCompute`
4. Upload our data to a `Datastore`

## 3.1 Link to or create a Workspace

In [ ]:
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>"
)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## 3.2 Set up an Experiment and logging

In [43]:
# Make a folder for the project
project_folder = './automl-sentence-similarity'
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

# Set up an experiment
experiment_name = 'automl-sentence-similarity'
experiment = Experiment(ws, experiment_name)

#Add logging to our experiment
run = experiment.start_logging()

## 3.3 Link AmlCompute compute target

To use AzureML Pipelines we need to link a compute target as they can not be run locally (see [compute options](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#supported-compute-targets) for explanation of the different options). We will use an AmlCompute target in this example.

In [44]:
# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-06-19T17:10:43.157000+00:00', 'errors': None, 'creationTime': '2019-05-20T22:09:40.142683+00:00', 'modifiedTime': '2019-05-20T22:10:11.888950+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## 3.4 Upload data to Datastore

This step uploads our local data to a `Datastore` so that the data is accessible from the remote compute target and creates a `DataReference` to point to the location of the data on the Datastore. A DataStore is backed either by a Azure File Storage (default option) or Azure Blob Storage ([how to decide between these options](https://docs.microsoft.com/en-us/azure/storage/common/storage-decide-blobs-files-disks)) and data is made accessible by mounting or copying data to the compute target.

In [45]:
# Select a specific datastore or you can call ws.get_default_datastore()
datastore_name = 'workspacefilestore'
ds = ws.datastores[datastore_name]

# Upload files in data folder to the datastore
ds.upload(src_dir='./data', target_path='stsbenchmark_data', overwrite=True, show_progress=True)

Uploading ./data\dev.csv
Uploading ./data\test.csv
Uploading ./data\train.csv
Uploaded ./data\train.csv, 1 files out of an estimated total of 3
Uploaded ./data\dev.csv, 2 files out of an estimated total of 3
Uploaded ./data\test.csv, 3 files out of an estimated total of 3


$AZUREML_DATAREFERENCE_dbe7476178794853924424fdfbc4dcc1

We also set up a **DataReference** object that points to the data we just uploaded into the stsbenchmark_data folder. DataReference objects point to data that is accessible from a datastore and will be used an an input into our pipeline.

In [46]:
input_data = DataReference(datastore=ds, 
                           data_reference_name="stsbenchmark",
                           path_on_datastore='stsbenchmark_data/',
                           overwrite=False)

# 4. Create AzureML Pipeline

Now we set up our pipeline which is made of two steps:  
1. `PythonScriptStep`: takes each sentence pair from the data in the `Datastore` and concatenates the Google USE embeddings for each sentence into one vector. This step saves the embedding feature matrix back to our `Datastore` and uses a `PipelineData` object to represent this intermediate data.  
2. `AutoMLStep`: takes the intermediate data produced by the previous step and passes it to an `AutoMLConfig` which performs the automatic model selection

## 4.1 Set up run configuration file

First we set up a `RunConguration` object which configures the execution environment for an experiment (sets up the conda dependencies, etc.)

In [47]:
# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target

conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = aml.core.runconfig.DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
conda_run_config.environment.python.user_managed_dependencies = False

conda_run_config.environment.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'azureml-sdk', 'azureml-dataprep', 'azureml-train-automl==1.0.33'], 
                              conda_packages=['numpy', 'py-xgboost', 'pandas', 'tensorflow', 'tensorflow-hub', 'scikit-learn'], 
                              pin_sdk_version=False)

print('run config is ready')

run config is ready


## 4.2 PythonScriptStep

`PythonScriptStep` is a step which runs a user-defined Python script ([documentation](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep?view=azure-ml-py) here). In this `PythonScriptStep`, we will convert our sentences into a numerical representation in order to use them in our machine learning model. We will embed both sentences using the Google Universal Sentence Encoder (provided by tensorflow-hub) and concatenate their representations into a $1024$-dimensional vector to use as features for AutoML.

**Google Universal Sentence Encoder:**
We'll use a popular sentence encoder called Google Universal Sentence Encoder (see [original paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf)). Google provides two pretrained models based on different design goals: a Transformer model (targets high accuracy even if this reduces model complexity) and a Deep Averaging Network model (DAN; targets efficient inference). Both models are trained on a variety of web sources (Wikipedia, news, question-answers pages, and discussion forums) and produced 512-dimensional embeddings. This notebook utilizes the Transformer-based encoding model which can be downloaded [here](https://tfhub.dev/google/universal-sentence-encoder-large/3) because of its better performance relative to the DAN model on the STS Benchmark dataset (see Table 2 in Google Research's [paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf)). The Transformer model produces sentence embeddings using the "encoding sub-graph of the transformer architecture" (original architecture introduced [here](https://arxiv.org/abs/1706.03762)). "This sub-graph uses attention to compute context aware representations of words in a sentence that take into account both the ordering and identity of all the other workds. The context aware word representations are converted to a fixed length sentence encoding vector by computing the element-wise sum of the representations at each word position." The input to the model is lowercase PTB-tokenized strings and the model is designed to be useful for multiple different tasks by using multi-task learning. More details about the model can be found in the [paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf) by Google Research.

### 4.2.1 Define python script to run

Define the script (called embed.py) that the `PythonScriptStep` will execute:

In [48]:
%%writefile $project_folder/embed.py
import argparse
import os
import azureml.core
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
tf.logging.set_verbosity(tf.logging.ERROR) # reduce logging output

def google_encoder(dataset):
    """ Function that embeds sentences using the Google Universal
    Sentence Encoder pretrained model
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    emb1: 512-dimensional representation of sentence1
    emb2: 512-dimensional representation of sentence2
    """
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    # Apply embedding model and normalize the input
    sts_encode1 = tf.nn.l2_normalize(embedding_model(sts_input1), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embedding_model(sts_input2), axis=1)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        emb1, emb2 = session.run(
          [sts_encode1, sts_encode2],
          feed_dict={
              sts_input1: dataset['sentence1'],
              sts_input2: dataset['sentence2']
          })
    return emb1, emb2

def feature_engineering(dataset):
    """Extracts embedding features from the dataset and returns
    features and target in a dataframe
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    df: pandas dataframe with embedding features
    scores: list of target variables
    """
    google_USE_emb1, google_USE_emb2 = google_encoder(dataset)
    n_google = google_USE_emb1.shape[1] #length of the embeddings 
    df = np.concatenate((google_USE_emb1, google_USE_emb2), axis=1)
    names = ['USEEmb1_'+str(i) for i in range(n_google)]+['USEEmb2_'+str(i) for i in range(n_google)]
    df = pd.DataFrame(df, columns=names)
    return df, dataset['score']

def write_output(df, path, name):
    """Write dataframes to correct path"""
    os.makedirs(path, exist_ok=True)
    print("%s created" % path)
    df.to_csv(path + "/" + name, index=False)

#Parse arguments
parser = argparse.ArgumentParser()
parser.add_argument("--sentence_data", type=str)
parser.add_argument("--embedded_data", type=str)
args = parser.parse_args()

# Import the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embedding_model = hub.Module(module_url)

#Read data 
train = pd.read_csv(args.sentence_data + "/train.csv")
dev = pd.read_csv(args.sentence_data + "/dev.csv")

#Get Google USE features
training_data, training_scores = feature_engineering(train)
validation_data, validation_scores = feature_engineering(dev)

#Write out training data to Datastore
write_output(training_data, args.embedded_data, "X_train.csv")
write_output(pd.DataFrame(training_scores, columns=['score']), args.embedded_data, "y_train.csv")

#Write out validation data to Datastore
write_output(validation_data, args.embedded_data, "X_dev.csv")
write_output(pd.DataFrame(validation_scores, columns=['score']), args.embedded_data, "y_dev.csv")

Overwriting ./automl-sentence-similarity/embed.py


### 4.2.2 Create PipelineData object

`PipelineData` objects represent a piece of intermediate data in a pipeline. Generally they are produced by one step (as an output) and then consumed by the next step (as an input), introducing an implicit order between steps in a pipeline. We create a PipelineData object that can represent the data produced by our first pipeline step that will be consumed by our second pipeline step.

In [49]:
embedded_data = PipelineData("embedded_data", datastore=ds)

### 4.2.3 Create PythonScriptStep

This step defines the `PythonScriptStep`. We give the step a name, tell the step which python script to run (embed.py) and what directory that script is located in (source_directory). Note that the hash_paths parameter will be deprecated but currently is needed to check for any updates to the embed.py file.

We also link the compute target and run configuration that we made previously. Our input is the `DataReference` object (input_data) where our raw sentence data was uploaded and our ouput is the `PipelineData` object (embedded_data) where the embedded data produced by this step will be stored. These are also passed in as arguments so that we have access to the correct data paths.

In [50]:
embedStep = PythonScriptStep(
    name="Embed",
    script_name="embed.py", 
    arguments=["--embedded_data", embedded_data,
               "--sentence_data", input_data],
    inputs=[input_data],
    outputs=[embedded_data],
    compute_target=compute_target,
    runconfig = conda_run_config,
    hash_paths=["embed.py"],
    source_directory=project_folder,
    allow_reuse=True
)

## 4.3 AutoMLStep

`AutoMLStep` creates an AutoML step in a pipeline (see [documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlstep?view=azure-ml-py) and [basic example](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.ipynb)). When using AutoML on remote compute, rather than passing our data directly into the `AutoMLConfig` object as we did in the local example, we must define a get_data.py script with a get_data() function to pass as the data_script argument. This workflow can be used for both local and remote executions (see [details](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-auto-train-remote)). 



### 4.3.1 Define get_data script to load data

Define the get_data.py file and get_data() function that the `AutoMLStep` will execute to collect data. Note that we can directly access the path of the intermediate data (called embedded_data) through `os.environ['AZUREML_DATAREFERENCE_embedded_data']`. This is necessary because the AutoMLStep does not accept additional parameters like the PythonScriptStep does with `arguments`.

In [51]:
%%writefile $project_folder/get_data.py

import os
import pandas as pd

def get_data():
    """Function needed to load data for use on remote AutoML experiments"""
    X_train  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/X_train.csv")
    y_train  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/y_train.csv")
    X_dev  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/X_dev.csv")
    y_dev  = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_embedded_data'] + "/y_dev.csv")
    return { "X" : X_train.values, "y" : y_train.values.flatten(), "X_valid": X_dev.values, "y_valid": y_dev.values.flatten()}

Overwriting ./automl-sentence-similarity/get_data.py


### 4.3.2 Create AutoMLConfig object

Now, we specify the parameters for the `AutoMLConfig` class:

**task**  
AutoML supports the following base learners for the regression task: Elastic Net, Light GBM, Gradient Boosting, Decision Tree, K-nearest Neighbors, LARS Lasso, Stochastic Gradient Descent, Random Forest, Extremely Randomized Trees, XGBoost, DNN Regressor, Linear Regression. In addition, AutoML also supports two kinds of ensemble methods: voting (weighted average of the output of multiple base learners) and stacking (training a second "metalearner" which uses the base algorithms' predictions to predict the target variable). Specific base learners can be included or excluded in the parameters for the AutoMLConfig class (whitelist_models and blacklist_models) and the voting/stacking ensemble options can be specified as well (enable_voting_ensemble and enable_stack_ensemble)

**preprocess**  
AutoML also has advanced preprocessing methods, eliminating the need for users to perform this manually. Data is automatically scaled and normalized but an additional parameter in the AutoMLConfig class enables the use of more advanced techniques including imputation, generating additional features, transformations, word embeddings, etc. (full list found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-portal-experiments#preprocess)). Note that algorithm-specific preprocessing will be applied even if preprocess=False. 

**primary_metric**  
The regression metrics available are the following: Spearman Correlation (spearman_correlation), Normalized RMSE (normalized_root_mean_squared_error), Normalized MAE (normalized_mean_absolute_error), and R2 score (r2_score) 

**Constraints:**  
There is a cost_mode parameter to set cost prediction modes (see options [here](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlconfig?view=azure-ml-py)). To set constraints on time there are multiple parameters including experiment_exit_score (target score to exit the experiment after acheiving), experiment_timeout_minutes (maximum amount of time for all combined iterations), and iterations (total number of different algorithm and parameter combinations to try).

In [52]:
automl_settings = {
    "iteration_timeout_minutes": 5, #How long each iteration can take before moving on
    "iterations": 5,  #Number of algorithm options to try
    "primary_metric": 'spearman_correlation', #Metric to optimize
    "preprocess": True,  #Whether dataset preprocessing should be applied
    "verbosity": logging.INFO,
}
automl_config = AutoMLConfig(task = 'regression', #type of task: classification, regression or forecasting
                 debug_log = 'automl_errors.log',
                 path = project_folder,
                 compute_target=compute_target,
                 run_configuration=conda_run_config,
                 data_script = project_folder + "/get_data.py", #local path to script with get_data() function
                 **automl_settings)

### 4.3.3 Create AutoMLStep

Finally, we create `PipelineData` objects for the model data (our outputs) and then create the `AutoMLStep`. The `AutoMLStep` requires a `AutoMLConfig` object and we pass our intermediate data (embedded_data) in as the inputs. Again, note that the hash_paths parameter will be deprecated but currently is needed to check for any updates to the get_data.py file.

In [53]:
# Create PipelineData objects for tracking AutoML metrics 
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [54]:
automl_step = AutoMLStep(
    name='AutoML',
    automl_config=automl_config, #the AutoMLConfig object created previously
    inputs=[embedded_data], #inputs is the PipelineData that was the output of the previous pipeline step
    outputs=[metrics_data, model_data], #PipelineData objects to reference metric and model information
    hash_paths=["get_data.py"],
    allow_reuse=True)

# 5. Run Pipeline

Now we set up our pipeline which requires specifying our `Workspace` and the ordering of the steps that we created (steps parameter). We submit the pipeline and inspect the run details using a RunDetails widget. For remote runs, the execution of iterations is asynchronous.

In [55]:
#automl_step.run_after(embedStep)
pipeline = Pipeline(
    description="pipeline_embed_automl", #give a name for the pipeline
    workspace=ws,    
    steps=[embedStep, automl_step])

In [56]:
pipeline_run = experiment.submit(pipeline)

Created step Embed [ba6c6c5a][d271deed-bd3b-4e41-9814-29fc11e585b4], (This step is eligible to reuse a previous run's output)
Created step AutoML [4cc715de][cd685f96-c946-46df-a4c4-87a53b92c90e], (This step will run and generate new outputs)
Using data reference stsbenchmark for StepId [0c4ee4ad][e3340790-c54f-4147-8dd0-bcb80a9b7b46], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: 994a7673-8f48-42b9-9cfa-1a47bf70c304


In [ ]:
# Inspect the run details using the provided widget
RunDetails(pipeline_run).show()

![](pipelineWidget.png)

In [58]:
pipeline_run.wait_for_completion(show_output=True) #show console output while run is in progress

PipelineRunId: 994a7673-8f48-42b9-9cfa-1a47bf70c304
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/automl-sentence-similarity/runs/994a7673-8f48-42b9-9cfa-1a47bf70c304
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 9f99614d-6bc5-4c10-a121-b09afdc02c74
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/automl-sentence-similarity/runs/9f99614d-6bc5-4c10-a121-b09afdc02c74

StepRun(Embed) Execution Summary
StepRun( Embed ) Status: Finished
{'runId': '9f99614d-6bc5-4c10-a121-b09afdc02c74', 'target': 'gpucluster', 'status': 'Completed', 'startTimeUtc': '2019-06-19T17:11:05.931189Z', 'endTimeUtc': '2019-06-19T17:11:06.005469Z', 'properties': {'azureml.reusedrunid': 'f

Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/automl-sentence-similarity/runs/a3a6fa4b-d2dc-479d-be93-eea52e5c597c
StepRun( AutoML ) Status: NotStarted
StepRun( AutoML ) Status: Running

StepRun(AutoML) Execution Summary
StepRun( AutoML ) Status: Finished
{'runId': 'a3a6fa4b-d2dc-479d-be93-eea52e5c597c', 'target': 'gpucluster', 'status': 'Completed', 'startTimeUtc': '2019-06-19T17:17:00.446785Z', 'endTimeUtc': '2019-06-19T17:25:31.355471Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'bac07baf-2af3-4a06-aadf-aef17548794d', 'StepType': 'AutoMLStep', 'azureml.pipelinerunid': '994a7673-8f48-42b9-9cfa-1a47bf70c304', 'num_iterations': '5', 'training_type': 'TrainFull', 'acquisition_function': 'EI', 'metrics': 'accuracy', 'primary_metric': 'spearman_correlation', 'train_split': '0', 'MaxTimeSeconds': '300',

'Finished'

In [59]:
#Publish the pipeline
published_pipeline = pipeline.publish(
    name="Sentence_Similarity_Pipeline", 
    description="Sentence Similarity with Google USE Features")